In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.linear_model import LinearRegression

In [3]:
good_data = pd.read_csv("../../data/system_prompts/Prompt-Scores_Good-Property.csv")
bad_data = pd.read_csv("../../data/system_prompts/Prompt-Scores_Bad-Property.csv")
good_data

,Prompt,Catagory,Comments,llama-2-7b-chat/ARC_acc,llama-2-7b-chat/ARC_acc_no_error,llama-2-7b-chat/ARC_error,llama-2-7b-chat/MMLU_acc,llama-2-7b-chat/MMLU_acc_no_error,llama-2-7b-chat/MMLU_error,llama-2-7b-chat/HELLASWAG_acc,...,gemma-7b-it/HELLASWAG_error,c4ai-command-r-v01/HELLASWAG_acc,c4ai-command-r-v01/HELLASWAG_acc_no_error,c4ai-command-r-v01/HELLASWAG_error,Mistral-7B-Instruct-v0.2/ARC_acc,Mistral-7B-Instruct-v0.2/ARC_acc_no_error,Mistral-7B-Instruct-v0.2/ARC_error,Mistral-7B-Instruct-v0.2/HELLASWAG_acc,Mistral-7B-Instruct-v0.2/HELLASWAG_acc_no_error,Mistral-7B-Instruct-v0.2/HELLASWAG_error
0,"You are a contextual assistant, able to unders...",Good-Property,PropertyCount: 1,0.516212,0.516212,0,0.408221,0.408554,2,0.494,...,1,0.626,0.626,0,0.700512,0.705326,8,0.544,0.552284,15
1,"You are an assertive assistant, confidently ta...",Good-Property,PropertyCount: 1,0.512799,0.512799,0,0.409035,0.409202,1,0.487,...,1,0.635,0.635,0,0.690273,0.696813,11,0.539,0.548880,18
2,"You are an organized assistant, capable of kee...",Good-Property,PropertyCount: 1,0.515358,0.515358,0,0.404558,0.404888,2,0.463,...,3,0.640,0.640,0,0.689420,0.694158,8,0.537,0.546843,18
3,"You are a balanced assistant, able to juggle m...",Good-Property,PropertyCount: 1,0.505973,0.505973,0,0.402930,0.403094,1,0.480,...,1,0.639,0.639,0,0.703072,0.709122,10,0.531,0.540733,18
4,"You are a robust assistant, capable of handlin...",Good-Property,PropertyCount: 1,0.515358,0.515358,0,0.404965,0.405130,1,0.474,...,2,0.648,0.648,0,0.692833,0.698194,9,0.550,0.560081,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
300,"You are an anomaly-detecting, exemplary, analy...",Good-Property,PropertyCount: 30,0.497440,0.497440,0,0.404558,0.404888,2,0.448,...,0,0.640,0.640,0,0.668089,0.679098,19,0.533,0.542770,18
301,"You are a trailblazing, reliable, versatile, p...",Good-Property,PropertyCount: 30,0.517918,0.517918,0,0.411477,0.411477,0,0.479,...,1,0.663,0.663,0,0.689420,0.698358,15,0.541,0.549797,16
302,"You are an AI system that embodies progress, i...",Good-Property,PropertyCount: 30,0.521331,0.521331,0,0.402523,0.402523,0,0.452,...,2,0.654,0.654,0,0.686007,0.693701,13,0.538,0.547304,17
303,"You are a streamlined, state-of-the-art, world...",Good-Property,PropertyCount: 30,0.532423,0.532423,0,0.411477,0.411477,0,0.473,...,1,0.651,0.651,0,0.691126,0.695876,8,0.551,0.560529,17


In [4]:
f = open("../../data/prompt_generation/good_property.txt", "r")
good_properties = f.read().split(", ")
f.close()

f = open("../../data/prompt_generation/bad_property.txt", "r")
bad_properties = f.read().split(", ")
f.close()

In [5]:
def get_importance(col_name="llama-2-7b-chat/ARC_acc_no_error", isgood=True):
    if isgood:
        tmp_data = good_data
        tmp_properties = good_properties
    else:
        tmp_data = bad_data
        tmp_properties = bad_properties
    
    reg_x = []
    reg_y = tmp_data[col_name]

    for prompt in tmp_data["Prompt"]:
        reg_x_tmp = [0 for i in range(len(tmp_properties))]
        for idx in range(len(tmp_properties)):
            if tmp_properties[idx].lower() in prompt.lower():
                reg_x_tmp[idx] = 1
        reg_x.append(reg_x_tmp)

    reg_x = pd.DataFrame(data=np.array(reg_x), columns=tmp_properties)
    model = LinearRegression()
    model.fit(reg_x, reg_y)

    importance = list(zip(model.coef_, model.feature_names_in_))
    importance.sort(reverse=True)
    return importance

In [6]:
good_data.columns

Index(['Prompt', 'Catagory', 'Comments', 'llama-2-7b-chat/ARC_acc',
       'llama-2-7b-chat/ARC_acc_no_error', 'llama-2-7b-chat/ARC_error',
       'llama-2-7b-chat/MMLU_acc', 'llama-2-7b-chat/MMLU_acc_no_error',
       'llama-2-7b-chat/MMLU_error', 'llama-2-7b-chat/HELLASWAG_acc',
       'llama-2-7b-chat/HELLASWAG_acc_no_error',
       'llama-2-7b-chat/HELLASWAG_error', 'Mixtral-8x7B-Instruct-v0.1/ARC_acc',
       'Mixtral-8x7B-Instruct-v0.1/ARC_acc_no_error',
       'Mixtral-8x7B-Instruct-v0.1/ARC_error',
       'Mixtral-8x7B-Instruct-v0.1/HELLASWAG_acc',
       'Mixtral-8x7B-Instruct-v0.1/HELLASWAG_acc_no_error',
       'Mixtral-8x7B-Instruct-v0.1/HELLASWAG_error',
       'Mixtral-8x7B-Instruct-v0.1/MMLU_acc',
       'Mixtral-8x7B-Instruct-v0.1/MMLU_acc_no_error',
       'Mixtral-8x7B-Instruct-v0.1/MMLU_error', 'gemma-7b/ARC_acc',
       'gemma-7b/ARC_acc_no_error', 'gemma-7b/ARC_error', 'gemma-7b/MMLU_acc',
       'gemma-7b/MMLU_acc_no_error', 'gemma-7b/MMLU_error',
       'gemma-7b

In [50]:
get_importance(col_name="llama-2-7b-chat/ARC_acc_no_error", isgood=True)

[(0.015332190264350256, 'Unsurpassed'),
 (0.013046993563809678, 'Incredible'),
 (0.01107242009030525, 'Analytical'),
 (0.010244172307389453, 'Excellent'),
 (0.007867198985639133, 'Empowering'),
 (0.007450636186485456, 'Respectful'),
 (0.007339253140260471, 'Enthusiastic'),
 (0.006875086493749859, 'Wise'),
 (0.006333989857723913, 'Disciplined'),
 (0.0058342227313139036, 'Perfected'),
 (0.0057356671043977195, 'Trustworthy'),
 (0.0057049741948034804, 'Proficient'),
 (0.00566042856812493, 'Compassionate'),
 (0.005521704626924719, 'Self-updating'),
 (0.005411399967488099, 'Dynamic'),
 (0.005397008032234005, 'Methodical'),
 (0.005356735971175213, 'State-of-the-art'),
 (0.005287512070708064, 'Robust'),
 (0.004938554075637495, 'Outstanding'),
 (0.0048982790926852884, 'Unrivaled'),
 (0.004799337349419967, 'Error-tolerant'),
 (0.004724187436820842, 'Precise'),
 (0.0047227308278258795, 'High-performing'),
 (0.004381038470683335, 'Humble'),
 (0.004104562859054303, 'Human-like'),
 (0.00392662144407

In [51]:
get_importance(col_name="llama-2-7b-chat/MMLU_acc_no_error", isgood=True)

[(0.011288805318120123, 'Elite'),
 (0.010872333675187813, 'Phenomenal'),
 (0.009801780369798403, 'Proficient'),
 (0.007468346510909555, 'Altruistic'),
 (0.00739843955700654, 'Adaptive'),
 (0.007338395140383663, 'Intelligent'),
 (0.006745753854996865, 'Respectful'),
 (0.006584970583428956, 'Utmost'),
 (0.006143512261682257, 'Customizable'),
 (0.005881869149188031, 'Wise'),
 (0.005658010062794316, 'Disciplined'),
 (0.005488703977814372, 'Cognitive'),
 (0.005477115108998494, 'Resilient'),
 (0.005443955975298837, 'Scalable'),
 (0.005326293978847299, 'Outstanding'),
 (0.005272011481807526, 'Incredible'),
 (0.005015191284719202, 'Humble'),
 (0.004990831105807953, 'Non-biased'),
 (0.004980831614432546, 'World-class'),
 (0.004934730273382922, 'High-performing'),
 (0.004760246158251028, 'Optimal'),
 (0.00458037279184765, 'Inclusive'),
 (0.004571756151001876, 'Unsurpassed'),
 (0.004563585343011138, 'State-of-the-art'),
 (0.004458962724096891, 'Cooperative'),
 (0.004272763876473151, 'Balanced'),


# Llama Different Benchmark scores (Bad)

# Llama Different Benchmark errors (Bad)

In [48]:
get_importance(col_name="llama-2-7b-chat/ARC_error", isgood=False)

[(261.66414244517034, 'Discriminatory'),
 (155.17101276303453, 'Offensive'),
 (137.55535360732463, 'Biased'),
 (104.346897826981, 'Displeasing'),
 (92.65442572520391, 'Provocative'),
 (89.31522533279004, 'Condescending'),
 (86.78949032256442, 'Ridiculous'),
 (83.92164864836437, 'Unfounded'),
 (83.40223522848419, 'Dated'),
 (75.91195872483092, 'Illogical'),
 (70.79864805317146, 'Disconnected'),
 (68.33042790985147, 'Stressful'),
 (60.367003301567806, 'Demeaning'),
 (60.090111769153665, 'Nasty'),
 (57.30340151906216, 'Shameful'),
 (51.983446048046446, 'Agitating'),
 (51.90252719886034, 'Frightening'),
 (50.29057916715894, 'Sensational'),
 (49.207577593682636, 'Energy-sapping'),
 (43.84727303079921, 'Upsetting'),
 (39.66631566235347, 'Unimportant'),
 (39.318570836927805, 'Shocking'),
 (38.12719012992872, 'Difficult'),
 (37.86034163071564, 'Foul'),
 (37.28158748277319, 'Non-intuitive'),
 (36.98002305036336, 'Unsolvable'),
 (35.66818102516862, 'Embarrassing'),
 (35.11528058303745, 'Callous'

In [49]:
get_importance(col_name="llama-2-7b-chat/MMLU_error", isgood=False)

[(440.54168135815036, 'Discriminatory'),
 (226.8947557008869, 'Biased'),
 (194.61890529294652, 'Offensive'),
 (171.26107166163652, 'Stressful'),
 (152.78600185464427, 'Unfounded'),
 (144.43507162943712, 'Displeasing'),
 (132.9875194116252, 'Ridiculous'),
 (122.62956144654348, 'Provocative'),
 (120.88317375495983, 'Energy-sapping'),
 (109.00286546688311, 'Demeaning'),
 (104.38602744085426, 'Illogical'),
 (91.07892607592754, 'Shameful'),
 (90.85054963347802, 'Disconnected'),
 (87.51196716582047, 'Dated'),
 (84.55281132496688, 'Condescending'),
 (84.0447966595359, 'Sensational'),
 (80.58187565607341, 'Upsetting'),
 (76.03008037592018, 'Foul'),
 (69.53232746957005, 'Unstable'),
 (69.29501614736425, 'Mystifying'),
 (68.07322831407406, 'Frightening'),
 (60.807950166842986, 'Preposterous'),
 (58.00985375920976, 'Baseless'),
 (51.58456212740375, 'Unethical'),
 (51.37790618101273, 'Non-intuitive'),
 (50.98254345207767, 'Unsolvable'),
 (49.98348013394695, 'Unimportant'),
 (49.98315596005047, 'Na

In [7]:
get_importance(col_name="llama-2-7b-chat/HELLASWAG_error", isgood=False)

[(151.61059722397647, 'Discriminatory'),
 (74.31229378344406, 'Stressful'),
 (70.89496307952834, 'Biased'),
 (47.76646050946213, 'Ridiculous'),
 (46.035128391104884, 'Offensive'),
 (45.6204892753924, 'Deflating'),
 (41.96747154541526, 'Demeaning'),
 (30.84746953465233, 'Displeasing'),
 (29.25106827581728, 'Dated'),
 (28.811389004919487, 'Condescending'),
 (27.51473778564516, 'Mystifying'),
 (26.600424939029644, 'Disconnected'),
 (24.165594550841718, 'Provocative'),
 (23.74062535915624, 'Energy-sapping'),
 (23.22293025847645, 'Antiquated'),
 (20.86479536639317, 'Illogical'),
 (20.217930325671894, 'Inconsistent'),
 (19.352435570084815, 'Sensational'),
 (18.957290520136098, 'Stuttering'),
 (18.10842376383079, 'Unsolvable'),
 (17.92375415838449, 'Preposterous'),
 (17.69716103216623, 'Confounding'),
 (17.48948800474972, 'Unimportant'),
 (17.478528960365136, 'Frightening'),
 (17.36108883696397, 'Unfounded'),
 (16.561754155052114, 'Unsubstantiated'),
 (14.65586442147793, 'Intrusive'),
 (14.58

In [ ]:
get_importance(col_name="llama-2-7b-chat/MMLU_error", isgood=False)